<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.incimages.com%2Fuploaded_files%2Fimage%2F1920x1080%2Fgetty_660952912_374375.jpg&f=1&nofb=1"  >

## Table of Contents
1. [Libraries](#library)
2. [How to read candlestick chart?](#candle)
3. [What are moving averages](#ma)
    - [Golden vs Death Cross](#50-200ma)
    - [Exponential Moving Average](#ema)
4. [MACD Strategy](#macd)
5. [52-Week High and Low](#highlow)
6. [LSTM Model](#lstm)
    - [Data Preprocessing](#preprocess)
    - [Model Configuration](#model)
    - [Computational Graph](#graph)
    - [Training and Calculate loss](#train)
    - [Testing on Data](#test)
7. [Bonus: Price Action](#priceaction)
8. [References](#references)


<a id="library"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Import Libraries
</h1>
</div>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
!pip install pandas_ta;
import pandas_ta as ta;
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
!pip install torchviz
from torchviz import make_dot

In [ ]:
fb = pd.read_csv('/kaggle/input/facebook-stock-data/FB.csv',parse_dates=['Date'])
fb.head(3)

<a id="candle"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> How to read Candlestick Chart ?
</h1>
</div>

![image.png](https://www.stockoptionstutorials.com/wp-content/uploads/2018/07/CANDLESTICKSbullbear.png)


<h5 style = "font-family:Monospace; font-size:30px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Bullish Candlestick</h5>

* <p style= " font-size:15px;
 font-family:georgia,serif;
 margin:10px;
 padding:10px;">Bullish candlestick is a formation that mark the presence of a market momentum dominated by <b>Buyers</b>.<br>
    Where :</p>
<p style="color: green; font-size: 15px; font-family: 'Signika', sans-serif; padding-bottom: 10px; font-weight:bold;"> Closing Price > Opening Price </p>

<h5 style = "font-family:Monospace; font-size:30px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Bearish Candlestick</h5>

* <p style= " font-size:15px;
 font-family:georgia,serif;
 margin:10px;
 padding:10px;">Bearish candlestick is a formation that mark the presence of a market momentum dominated by <b>Sellers</b>.<br>
    Where :</p>

<p style="color: red;  font-size:15px;
 font-family:georgia,serif;
 margin:10px;
 padding:10px; font-weight:bold;"> Opening Price > Closing Price </p>

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.09, subplot_titles=('Candlestick Chart', 'Volume'), 
               row_width=[0.2, 0.7])


fig.add_trace(go.Candlestick(x=fb["Date"], open=fb["Open"], high=fb["High"],
                low=fb["Low"], close=fb["Close"], name="OHLC"), 
                row=1, col=1
)

fig.add_trace(go.Bar(x=fb['Date'], y=fb['Volume'], showlegend=True,opacity=1,marker_line_color='green',
                     marker_line_width=2), row=2, col=1)



#time frame selector
fig.update_xaxes( row=1,col=1,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
   
)
fig.update_yaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True)


fig.update(layout_xaxis_rangeslider_visible=False)

fig.update_layout(title='Growth Of Facebook Stock over the years',
                  yaxis_title='Price',
                  showlegend=False,
                  paper_bgcolor='#efefef',
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
                 )


fig.show()

<h5 style = "font-family:Monospace; font-size:35px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Let's Begin with the Exploratory "Technical" Analysis</h5>

<a id="ma"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> What are moving averages ?
</h1>
</div>

<p style="
 font-size:15px;
 font-family:georgia,serif;
 margin:10px;
 padding:10px;">Moving averages are technical tools used on charts for trend identification.<br>They
can be used as standalone lines for trend trading, or in conjunction with other
technical indicators like the MACD, RSI, price levels of support and resistance,
or even with other moving average <br>
<br>
A simple moving average (SMA) is an indicator that shows a line on a chart
based on the calculation of the average price of a trading instrument over a set
number of time periods. A 5 day simple moving average is the average price
over a 5 day period. If something ended the last five days at \$100, \$101, \$99,
\$98, and \$102,the five day moving average would be $100.
<br><br>
<b>\$100+\$101+\$99+\$98+\$102= \$500/5= \$100.</b>
<br><br>
For Example : The 200-day moving average is represented as a line on charts and represents the average price over the past 200 days or 40 weeks.
</p>

In [ ]:
fb['MA50'] = fb.Close.rolling(50).mean()
fb['MA200'] = fb.Close.rolling(200).mean()


# plot the candlesticks
fig = go.Figure(data=[go.Candlestick(x=fb.Date,
                                     open=fb.Open, 
                                     high=fb.High,
                                     low=fb.Low,
                                     close=fb.Close,
                                    showlegend=False), 
                      go.Scatter(x=fb.Date, y=fb.MA50, line=dict(color='green', width=1.5),name='50 MA'),
                      go.Scatter(x=fb.Date, y=fb.MA200, line=dict(color='black', width=1.5),name='200 MA')])

fig.update_layout(title='50-200 Day Moving average(DMA)',
                  yaxis_title='Price',
                  xaxis_title='Year',
                  plot_bgcolor='#efefef',
                  paper_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
    legend=dict( xanchor = "center",
                 x = 0.92,
                 y=0.1,

)
                 )
#time frame selector
fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_yaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True)

fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

<ul style="
 font-size:15px;
 font-family:georgia,serif;
 margin:5px;
 padding:5px;"><li><b style="color:green;">50 day MA:</b> This is the line that strong leading stocks typically pull back
to. This is usually the support level for strong uptrends. It is normal for
uptrending markets to pull back to this line and find support. Most bull
markets and uptrends will pull back to this level. <b>It is generally a great
“Buy the dip” level</b>..In simpler terms buy the stock  whenever price line touches/retraces to 50MA line.</li><br>
    
 <li><b>200 day MA:</b> Bulls like to buy dips when markets are trading above the
200  day  moving  average,  while  bears  sell  rallies  short  below  it. <b> Bears
usually win below this line</b>, as the 200 day becomes longer term resistance,
and bulls buy pullbacks to the 200 day as long as the price stays above it.
This line is one of the <b>biggest signals</b> in the market telling you which side
to be on. Bull above, Bear below. <b>Bad things happen to stocks and markets
     when this line is lost</b>.</li>
</ul>
<ul style="
 font-size:15px;
 font-family:georgia,serif;
 margin:5px;
 padding:5px;"><li>In our Example Facebook Stock, The Price is clearly above the 50 and 200 MA,shows the stock is currently outperforming in the market.Generally I would say it would be a <b>great buy for the short term trade for few percent of gains But I would avoid for the longer term since the value of the stock is much higher</b> and there is a significant difference/distance between the 200 MA and the current market price ,So it would not be a Buy the dip opportunity for long-term holding.I would wait for the price to comeback/trace back to near 200 MA line and that would be a good opportunity to accumulate the stock (Assuming the fundamentals are also strong)</li>
</ul>
<p style="
 font-size:15px;          
 font-family:georgia,serif;
 margin:5px;
 padding:5px;"><b>January-March 2021 was a good opportunity to accumulate the stock as the price was near 200MA</b>.
</p>


<a id="50-200ma"></a>
<h5 style = "font-family:Monospace; font-size:30px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Golden Cross vs Death Cross</h5>

In [ ]:
fb3=fb[(fb.Date>='2018-07-01') & (fb.Date<='2019-10-31')]
# plot the candlesticks
fig = go.Figure(data=[go.Candlestick(x=fb3.Date,
                                     open=fb3.Open, 
                                     high=fb3.High,
                                     low=fb3.Low,
                                     close=fb3.Close,
                                    showlegend=False), 
                      go.Scatter(x=fb3.Date, y=fb3.MA50, line=dict(color='green', width=1.5),name='50 MA'),
                      go.Scatter(x=fb3.Date, y=fb3.MA200, line=dict(color='red', width=1.5),name='200 MA')])

fig.update_layout(title='Golden Cross & Death Cross',
                  yaxis_title='Price',
                  xaxis_title='Year',
                     plot_bgcolor='#efefef',
                  paper_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
     legend=dict( xanchor = "center",
                 x = 0.95,
                 y=0.1,

)                 
                 )

#add anotate - death cross 
fig.add_annotation(x='2018-09-20', y=180.653899,
            text="Death Cross",
            showarrow=True,
            arrowhead=1,
            xref="x1",
            yref="y1",
            font=dict(
                family="Courier New, monospace",
                size=20,
                color='red'
                    ),
            textangle=0,
            align="center",
            arrowsize=1,
            yanchor="bottom",
            xanchor="left",
            arrowwidth=2,
#             arrowcolor="#636363",
            ax=30,
            ay=-40,
            opacity=1
                  )
fig.add_annotation(x='2019-04-02', y=163.7096,
            text="Golden Cross",
            showarrow=True,
            arrowhead=1,
            xref="x1",
            yref="y1",
            font=dict(
                family="Courier New, monospace",
                size=20,
                color='green'
                    ),
            textangle=0,
            align="center",
            arrowsize=1,
            yanchor="top",
            xanchor="left",
            arrowwidth=2,
            arrowcolor="green",
            ax=30,
            ay=40,
            opacity=1
                  )

fig.update_yaxes(dtick=10,gridcolor='Lightgrey',showgrid=True,zeroline=False)
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True)
fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

<h3 style=" font-family:georgia,serif;">Golden Cross</h3>
<ul style="list-style-type:disc;
font-size:15px;          
 font-family:georgia,serif;
 margin:5px;
 padding:5px;">
    <li>A Golden Cross occurs when the <b style="color:green;">short-term moving average(50 MA)</b> crosses the <b style="color:red;">long-term moving average(200 MA)</b> from the <b>below to the above </b>signaling a confirmation of the <b>bullish</b> trend and an acceleration to the upside.
    </li>
</ul>

<h3 style=" font-family:georgia,serif;">Death Cross</h3>
<ul style="list-style-type:disc;
            font-size:15px;
 font-family:georgia,serif;
 margin:5px;
           padding:5px;">
    <li>A Death Cross occurs when the <b style="color:green;">short-term moving average(50 MA)</b> crosses the <b style="color:red;">long-term moving average(200 MA)</b> from the <b>above to the below</b> signaling a confirmation of the <b>bearish</b> trend and an acceleration to the downside.
    </li>
</ul>

<a id="ema"></a>
<h5 style = "font-family:Monospace; font-size:30px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Exponential Moving Average (EMA )</h5>

![image.png](https://cdn.educba.com/academy/wp-content/uploads/2019/12/Exponential-Moving-Average-Formula.jpg.webp)

<ul style="
 font-size:15px;
 font-family:georgia,serif;
 margin:5px;
           padding:5px;"><li>EMAs place a <b>higher weighting</b> on recent data than on older data, they are more reactive to the latest price changes than SMAs are, which makes the results from EMAs more timely and explains why the <b> EMA is the preferred average</b> among many traders. 
    </li></ul>

In [ ]:
fb['EMA50'] = ta.ema(fb['Close'], 50)
fb['EMA20'] = ta.ema(fb['Close'], 200)

# plot the candlesticks
fig = go.Figure(data=[go.Candlestick(x=fb.Date,
                                     open=fb.Open, 
                                     high=fb.High,
                                     low=fb.Low,
                                     close=fb.Close,showlegend=False), 
                      go.Scatter(x=fb.Date, y=fb.EMA50, line=dict(color='green', width=1.8),name='50 EMA'),
                      go.Scatter(x=fb.Date, y=fb.EMA20, line=dict(color='black', width=1.8),name='20 EMA')])

fig.update_layout(title='20-50 EMA',
                  yaxis_title='Price',
                  xaxis_title='Year',
                  plot_bgcolor='#efefef',
                  paper_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
                   legend=dict( xanchor = "center",
                 x = 0.95,
                 y=0.1,

)
                 )
#time frame selector
fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_yaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True,)
fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()
    

<a id="macd"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> MACD Strategy
</h1>
</div>

<p style="
 font-size:15px;          
 font-family:georgia,serif;
 margin:5px;
 padding:5px;">
    Moving Average Convergence Divergence(MACD) is a trend-following momentum indicator that showcases potential reversals, momentum and helps determine the trend.
</p>

<p style="
 font-size:15px;          
 font-family:georgia,serif;
 margin:5px;
 padding:5px;">
 <ul>
     <li><b style="color:green;">MACD Line(Green Line):</b>It's the difference between the 12-period EMA and the 26 period EMA.</li>
     <li><b style="color:red;">Signal Line(Red Line):</b> The Signal line is the 9-period EMA of MACD Line.</li>
     <li><b>MACD Historgram(Red/Green Bars):</b>It Simply represents the difference between the MACD line and the signal line. THe bigger the gap between the lines, the higher the bars that the MACD histogram will display.</li>
<hr>
</ul>
<h3 style="font-family:georgia,serif;">Understanding the MACD</h3>
<ul style="list-style-type:disc;">
    <li>When the <b style="color:green;">MACD line </b> goes above the <b style="color:red;"> the signal line</b> that means that the stocks momentum is increasing.</li><li> When the <b style="color:red;"> Signal line </b> goes above the <b style="color:green;"> the MACD line</b> that means that the stocks momentum is decreasing.</li>
    <li> When the <b style="color:green;"> MACD line </b> crosses above the <b style="color:orange;"> the zero line</b> this signals an uptrend.</li>
    <li> When the <b style="color:green;">MACD line </b> crosses below the <b style="color:orange;"> the zero line</b> this signals an downtrend.</li>
     
</ul>
<h5 style="font-family:georgia,serif;"> <b> NOTE: All these condiditons are possiblity not a necessary.</b></h5>
<hr>
<h3 style="font-family:georgia,serif;">Understanding the Histogram</h3>
<ul>
    <li>
        When the<b style="color:green;"> MACD line </b> goes above the <b style="color:red;"> Signal line</b> Histogram becomes Green. 
    </li>
    <li>
        When the<b style="color:red;"> Signal line </b> goes above the <b style="color:green;"> MACD line</b> Histogram becomes Red. 
    </li>
    <li>
        Wider the gap between the<b style="color:red;"> Signal line </b> and  the <b style="color:green;"> MACD line</b> ,the larger the histogram bar gets. 
    </li>
</ul>
</p>

In [ ]:
fb2=fb.tail(300).copy()
k = fb2['Close'].ewm(span=12, adjust=False, min_periods=12).mean()
# Get the 12-day EMA of the closing price
d = fb2['Close'].ewm(span=26, adjust=False, min_periods=26).mean()
# Subtract the 26-day EMA from the 12-Day EMA to get the MACD
macd = k - d
# Get the 9-Day EMA of the MACD for the Trigger line
macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
# Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
macd_h = macd - macd_s
# Add all of our new values for the MACD to the dataframe
fb2['macd'] = fb2.index.map(macd)
fb2['macd_h'] = fb2.index.map(macd_h)
fb2['macd_s'] = fb2.index.map(macd_s)


# fb2.tail(254).ta.macd(close='Close', fast=12, slow=26, signal=9,append=True)

fig = make_subplots(rows=2, cols=1,shared_xaxes=True,vertical_spacing=0.06,subplot_titles=('Candlestick Chart', 'MACD Indicator'))
# price Line
# fig.append_trace(
#     go.Scatter(
#         x=fb.Date,
#         y=fb['Open'],
#         line=dict(color='#ff9900', width=1),
#         name='Open',
#         # showlegend=False,
#         legendgroup='1',
#     ), row=1, col=1
# )
# Candlestick chart for pricing
fig.append_trace(
    go.Candlestick(
        x=fb2.Date,
        open=fb2['Open'],
        high=fb2['High'],
        low=fb2['Low'],
        close=fb2['Close'],
        showlegend=False,
    ), row=1, col=1
)
# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=fb2.Date,
        y=fb2['macd'],
        line=dict(color='green', width=2),
        name='macd',
        # showlegend=False,
        legendgroup='2',
    ), row=2, col=1
)
# Slow signal (%d)
fig.append_trace(
    go.Scatter(
        x=fb2.Date,
        y=fb2['macd_s'],
        line=dict(color='red', width=2),
        # showlegend=False,
        legendgroup='2',
        name='signal',
        
    ), row=2, col=1
)
#1st- buy signal
fig.add_annotation(x='2020-07-31', y=0.6587852,
            text="Buy",
            showarrow=True,
            arrowhead=1,
            xref="x2",
            yref="y2",
            font=dict(
                family="Courier New, monospace",
                size=16,
                    ),
            textangle=0,
            align="center",
            arrowsize=1,
            yanchor="top",
            xanchor="right",
            arrowwidth=2,
#             arrowcolor="#636363",
            ax=-20,
            ay=20,
            opacity=1
                  )

#1st - sell signal
fig.add_annotation(x='2020-09-04', y=11.487061,
            text="Sell",
            showarrow=True,
            arrowhead=1,
            xref="x2",
            yref="y2",
            font=dict(
                family="Courier New, monospace",
                size=16,
                    ),
            textangle=20,
            align="center",
            arrowsize=1,
            yanchor="top",
            xanchor="right",
            arrowwidth=2,
#             arrowcolor="#636363",
            ax=-20,
            ay=0,
            opacity=1
                  )

#2nd- buy signal
fig.add_annotation(x='2021-03-10', y=-1.91125,
            text="Buy",
            showarrow=True,
            arrowhead=1,
            xref="x2",
            yref="y2",
            font=dict(
                family="Courier New, monospace",
                size=16,
                    ),
            textangle=0,
            align="center",
            arrowsize=1,
            yanchor="top",
            xanchor="right",
            arrowwidth=2,
#             arrowcolor="#636363",
            ax=-20,
            ay=20,
            opacity=1
                  )
#2nd- Sell signal
fig.add_annotation(x='2021-04-16', y=9.35975,
            text="Sell",
            showarrow=True,
            arrowhead=1,
            xref="x2",
            yref="y2",
            font=dict(
                family="Courier New, monospace",
                size=16,
                    ),
            textangle=0,
            align="center",
            arrowsize=1,
            yanchor="top",
            xanchor="left",
            arrowwidth=2,
#             arrowcolor="#636363",
            ax=30,
            ay=-20,
            opacity=1
                  )

# Colorize the histogram values
colors = np.where(fb2['macd_h'] < 0, 'green', 'red')
# Plot the histogram
fig.append_trace(
    go.Bar(
        x=fb2.Date,
        y=fb2['macd_h'],
        name='histogram',
        marker_color=colors,
    ), row=2, col=1
)
# Make it pretty
layout = go.Layout(title='MACD Strategy',
    yaxis_title="Price",
    plot_bgcolor='#efefef',
    paper_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)
# Update options and show plot
fig.update_layout(layout)
fig.update_yaxes(gridcolor='Lightgrey',showgrid=True,zerolinecolor='orange')
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True)
fig.show()

<div class="alert alert-block alert-info" style="font-size:15px; font-family:verdana; color:black;color:black;">
    📌 Don’t base your trading decisions primarily on indicators and their signals. Trend-following indicators will return a buy signal when prices start to move higher, even if the market is trading sideways. Similarly, oscillators and momentum indicators will give you a selling signal when prices start to rise during an uptrend.

There is no single best indicator, which is why you should combine different types of indicators and incorporate them into a broader trading strategy.
</div>

<a id="highlow"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> 52 Week High and Low of Facebook
</h1>
</div>

<p style="
 font-size:15px;          
 font-family:georgia,serif;
 margin:5px;
 padding:5px;">The 52-Week High/Low indicates a stock is trading at its highest or lowest price in the past 52 weeks. This is an important indicator for many investors in determining the current value of a stock or predicting a trend in a stock’s performance.
</p>


In [ ]:
fb1=fb[fb.Date>='2020-08-06']
print("52-week High :")

# fb1[fb1.High==fb1.High.max()][['Date','High']]
high_date = fb1[fb1.High==fb1.High.max()]['Date'].to_string()[7:]
high_price = fb1[fb1.High==fb1.High.max()]['High'].values[0]

print(f"Date : {high_date} , High :{high_price}")

# fb1[fb1.Low==fb1.Low.min()][['Date','Low']]

print("52-week Low :")
low_date=fb1[fb1.Low==fb1.Low.min()]['Date'].to_string()[7:]
low_price=fb1[fb1.Low==fb1.Low.min()]['Low'].values[0]

print(f"Date : {low_date} , Low :{low_price}")

In [ ]:
fig = go.Figure()
# plot scatter
fig.add_trace(go.Scatter(
    x=fb1.Date,
    y=fb1.Close,
    line=dict(color='black'),
))
# Low text
fig.add_annotation(
        x='2020-09-21',
        y=fb1.Low.min(),
        xref="x",
        yref="y",
        text="Low",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=20
            ),
        textangle=0,
        align="center",
        arrowhead=1,
        arrowsize=1,
        yanchor="top",
        xanchor="left",
        arrowwidth=2,
        ax=20,
        ay=10,
        )
#High text
fig.add_annotation(
        x='2021-07-28',
        y=fb1.High.max(),
        xref="x",
        yref="y",
        text="High",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=20,
            ),
        textangle=0,
        align="center",
        arrowhead=1,
        arrowsize=1,
        yanchor="bottom",
        xanchor="right",
        arrowwidth=2,
        ax=-20,
        ay=-20
        )
fig.update_layout(showlegend=False,
                  
                  title='52-Week High & Low ',
                  yaxis_title='Price',
                  xaxis_title='Date',
                  plot_bgcolor='#efefef',
                  paper_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    ))
fig.update_yaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True,)

fig.show()

<a id="lstm"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> LSTM Model
</h1>
</div>

<h5 style = "font-family:Monospace; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left">RNN Simplified</h5>
<h4 style=" font-family:georgia,serif;">How RNN(Recurrent Neural Network) is Different from ANN(Artifical Neural Network) ??</h4>
<ul style="list-style-type:disc;
font-size:15px;          
 font-family:georgia,serif;
 margin:5px;
 padding:5px;">
    <li>In Simple terms, ANN is a type of Neural network where the output state(Labels) is dependent on current input state(Features) . The information/data flows in forward manner(FeedForward) and there is a hidden layer between an input and output state. That hidden layer could be an Non linear function (Activation Functions).
    </li><hr>
     <li>Whereas, RNN is a type of Neural network where the output state(Labels) is dependent on  current input state(Features) as well as previous input state(previous outcome) . The information/data flows not only in forward direction but also act as a feedback loop  and  the hidden layer produces two outputs one of them goes to the next time step.<br><br>
         RNN has an advantage over ANN where it remembers each information through time .
         <br><br>
         Disadvantage  of RNN is they have vanishing gradient and exploding problems and this is where LSTM comes in picture as they can remember long and relevant information through time and forgets the information which is not important according to the usecase 
    </li>
    
</ul>

<a href="https://ibb.co/r4Q1RzV"><img src="https://i.ibb.co/HnKkfbj/Screenshot-2021-08-23-at-17-23-26-Microsoft-Whiteboard.png" alt="Screenshot-2021-08-23-at-17-23-26-Microsoft-Whiteboard" border="0"></a>


<h4 style="list-style-type:disc;
font-size:15px;          
 font-family:georgia,serif;
 margin:5px;
 padding:5px;"> For more in-depth understanding of RNN and LSTM ,Please refer this amazing blog by christopherolah <a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/"> :Link </a> </h4>
 
 


<a id="preprocess"></a>
<h5 style = "font-family:Monospace; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Data Preprocessing</h5>

In [ ]:
#LSTM are sensitive to our scale data therefore it would be nice if we scale our data b/w 0 to 1
def minmax_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

#create a window that takes sequential data and sequence length suppose 13 ..so it would lookback price of 13 days and predict the price of 14th day 
# and then we slide/shift the window by 1 day and it returns trainX data and trainY
def window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i + seq_length, [-1]]  # Next close price
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#choosing /initializing hypermeters
seq_length = 7
data_dim = 1
hidden_dim = 32
output_dim = 1
learning_rate = 0.01
epochs = 500

# create an array of close prices of the stock 
xy = fb.iloc[:,3:4].values

#spliting the data into train and test set ,70% and 30% respectively ( without shuffle)
train_size = int(len(xy) * 0.7)
train_set = xy[0:train_size]
test_set = xy[train_size - seq_length:]

#scaling the train_Set and test_set
train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

#applying window function 
trainX, trainY = window(train_set, seq_length)
testX, testY = window(test_set, seq_length)

#check if the device can be switched to gpu if not then cpu 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)

# converting the data from nparray to tensor and sending it to gpu(if available)
trainX_tensor = torch.FloatTensor(trainX).to(device)
trainY_tensor = torch.FloatTensor(trainY).to(device)

testX_tensor = torch.FloatTensor(testX).to(device)
testY_tensor = torch.FloatTensor(testY).to(device)

<a id="model"></a>
<h5 style = "font-family:Monospace; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left">Model Configuration</h5>

In [ ]:

# Our lstm model configuration
class model(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(model, self).__init__()
         # Define the LSTM layer
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        # Define the output layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

    def forward(self, x):
        # Forward pass through LSTM layer
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1])
        return x


<h5 style = "font-family:Monospace; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Initialize Model,Loss,Optimizer</h5>

In [ ]:
#initialize the model and send it to gpu where our data lies.
model = model(data_dim, hidden_dim, output_dim, 1).to(device)

# loss & optimizer setting
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# create losses list so that we can append loss of each epoch into the list
losses = np.zeros(epochs)

<a id="graph"></a>
<h5 style = "font-family:Monospace; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Pytorch Computational Graph Of LSTM Model</h5>

In [ ]:
yhat = model(trainX_tensor)

#visualize using makedot
make_dot(yhat, params=dict(model.named_parameters()))


<a id="train"></a>
<h5 style = "font-family:Monospace; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Training and Calculate Loss</h5>

In [ ]:
#training loop
for i in range(epochs):
    # zero the gradients as we dont want to accumulate it.
    optimizer.zero_grad()
    #calculate yhats/outputs
    outputs = model(trainX_tensor)
    #calculate loss using mse
    loss = criterion(outputs, trainY_tensor)
    #calculating gradients
    loss.backward()
    #updating the parameters using adam optim
    optimizer.step()
    #append the loss of i th epoch to list of losses
    losses[i] = loss.item()
    #print losses per 50 epoch
    if (i+1) % 50 == 0:
        print(f'epoch {i}, training loss : {loss.item()}')

In [ ]:
Final_loss = losses[-1]
fig = px.line(losses)
fig.update_layout(showlegend=False,
                  title='Training Loss',
                  yaxis_title=' Loss',
                  xaxis_title='Epoch',
                  plot_bgcolor='#efefef',
                  paper_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    ),
    legend=dict( xanchor = "center",
                 x = 0.95,
                 y=0.1,

))

fig.add_annotation(
        x=490,
        y=0.02,
        xref="x",
        yref="y",
        text=f"Loss :{Final_loss:.9f}",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=20,
            ),
        textangle=0,
        align="center",
        arrowhead=1,
        arrowsize=1,
        yanchor="bottom",
        xanchor="right",
        arrowwidth=2,
        ax=-20,
        ay=-20
        )
fig.update_yaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)

fig.show()


<a id="test"></a>
<h5 style = "font-family:Monospace; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Testing on Data</h5>

In [ ]:
#calculate outputs and get back it into cpu from gpu and convert it into nparray from tensor
pred_test = model(testX_tensor).cpu().detach().numpy()

pred_test1 = pd.DataFrame(pred_test, columns=['pred_test'])
testY1 = pd.DataFrame(testY, columns=['pred_test'])
# print(pred_test.shape)
# print(testY.shape)
fig = go.Figure()


df = pd.merge(pred_test1, testY1, on=pred_test1.index, how='outer')
df.drop('key_0',axis=1,inplace=True)
df.head()

fig.add_trace(go.Scatter(x=df.index, y=df.pred_test_x,
                    mode='lines',
                    name='predictions'))
fig.add_trace(go.Scatter(x=df.index, y=df.pred_test_y,
                    mode='lines',
                    name='original'))
# fig = px.line(test)

fig.update_layout(showlegend=True,
                  title='Stock price Prediction ',
                  yaxis_title=' Standardized Price',
                  xaxis_title='Time',
                  plot_bgcolor='#efefef',
                  paper_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    ),
    legend=dict( xanchor = "center",
                 x = 0.95,
                 y=0.1,

))
fig.update_yaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)
fig.update_xaxes(gridcolor='Lightgrey',showgrid=True,zeroline=False)

fig.show()


<a id="priceaction"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> BONUS : Price Action 
</h1>
</div>

![priceactiong.png](https://www.tradingview.com/x/ks0NOW1O/)

<h5 style = "font-family:georgia,serif; font-size:20px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> The views expressed here are of my own and only for educational purpose . Do your own research before investing or Consult your Financial Advisor.</h5>

<a id="references"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> References
</h1>
</div>

1. [Moving Averages 101: Incredible Signals That Will Make You Money in the Stock Market ](https://www.amazon.com/Moving-Averages-101-Incredible-Signals-ebook/dp/B011HGUQQS)
2. [Calculating the MACD in Python](https://www.alpharithms.com/calculate-macd-python-272222/)
3. [Candlestick Charts in Python ](https://plotly.com/python/candlestick-charts/)
4. [Learn Technical Analysis by Rayner Teo](https://www.youtube.com/user/tradingwithrayner)
5. [Beautiful Notebooks : Formatting Tutorial](https://www.kaggle.com/shubhamksingh/create-beautiful-notebooks-formatting-tutorial)

<h5 style = "font-family:georgia,serif; font-size:35px; background-color: white; color : RebeccaPurple; border-radius: 100px 100px; text-align:left"> Please do upvote,If you found it helpful. <br>Thank you🤗</h5>